# Overview

In class this week, we went over how to create an event-related average (i.e., to compute the brain's average response to a given type of event, such as a stimulus or experimental condition). Here, we will compute event-related averages for several conditions in the same experiment, and visualize the average data that we create.

Here are (some of...) the tools from lecture that you will need for this assignment.

In [ ]:
import numpy as np
import neurods
import cortex
import os

In [ ]:
# Here is the load_data function we created (this will be in neurods next week)
import nibabel
from scipy.stats import zscore
def load_data(*files, do_zscore=False, mask=None, dtype=np.float32):
    """Load fMRI data from files and optionally z-normalize data
    
    Parameters
    ----------
    files : strings 
        Absolute path names for files to be loaded
    do_zscore : bool
        Flag that determines whether to zscore data in time or not
    mask : boolean array
        Selection mask that specifies which voxels to extract from 3D brain
    dtype : numpy data type
        Data type to which to convert the loaded data

    Returns
    -------
    data : array
        fMRI data array, in (time, z, y, x) format (if not masked) or in
        (time, voxels) format (if masked)
    """
    # Create a list to store data
    data = []
    # Loop over files in list
    for f in files:
        print("Loading {}...".format(f))
        nii = nibabel.load(f)
        tmp = nii.get_data().T.astype(dtype)
        # Optionally mask data
        if mask is not None:
            tmp = tmp[:, mask]
        # Optionally zscore each run independently
        if do_zscore:
            tmp = zscore(tmp, axis=0)
        data.append(tmp)
        del tmp
    # Concatenate full data
    data = np.vstack(data)
    return data

# And here is the get_onsets function:
def get_onsets(cond):
    """Convert a set of indicators for when a condition is on to onset indices for that condition
    
    Parameters
    ----------
    cond : array
        An array of 1s and 0s (or a boolean array of Trues and Falses), indicating which time indices of
        an experimental timecourse were part of a single given condition
    
    Returns
    -------
    onset_times : array
        onset time indices for `cond`
    """
    # (Note fancy syntax from above to pull out first element of a tuple)
    on_times, = np.nonzero(cond)
    keepers = np.diff(np.hstack([-1, on_times]))>1
    onset_times = on_times[keepers]
    return onset_times

Here is a quick example of how to load data (look to lecture too!)

In [ ]:
sub, xfm = 's01', 'catloc' # Subject 1 for category localizer experiment
# NOTE: added masking here at data loading
cortical_voxels = cortex.db.get_mask(sub, xfm, type='cortical')
data_file = os.path.join(neurods.io.data_list['fmri'],'categories','s01_categories_01.nii.gz')
data = load_data(data_file, do_zscore=True, mask=cortical_voxels)
design_file = os.path.join(neurods.io.data_list['fmri'],'categories','experiment_design.npz')
design = np.load(design_file)

1. Compute an event-related average of the 10 TRs following condition onset for each of the five conditions in the experiment. [4 pts]
2. View each event-related average as a movie, the same way as is done at the end of the lecture (see the `lecture_breakout.ipynb` notebook). Briefly describe what you see in each different event-related average.  [2 pts]
3. Use `cortex.quickflat.make_figure()` to make a plot of the average response at the 5th TR after condition onset for each condition. [2 pts]
4. Repeat (1) and (3) using all the data from all three runs of the category localizer experiment. How does the inclusion of more data change the results? (Does it change the results?) [2 pts]

**BONUS POINTS [2 pts]**

Write a function that will compute an event-related average given a data set, the experimental design array, a specified condition, and the number of time points 


In [ ]:
# What's in this variable again?
design.keys()

In [ ]:
### STUDENT ANSWER (part 1)
# Compute event-related averages for each condition
event_duration = 10
# Set up variable to store condition average(s)
# (These steps are not necessary for full points, but they make everything shorter / cleaner)
condition_names = design['conditions'].tolist()
# ... for raw average data
cond_avg_data = dict()
# ... for pycortex volumes of condition averages
cond_avg_vols = dict()
for cond_num, cond_name in enumerate(condition_names):
    # Get onset indices
    cond_bool = design['run1'][:, cond_num]
    onset_indices = neurods.utils.get_onsets(cond_bool)
    tmp = []
    for oi in onset_indices:
        tmp.append(data[oi:oi+event_duration, :])
    tmp = np.array(tmp)
    tmp = np.mean(tmp, axis=0)
    # Save average data
    cond_avg_data[cond_name] = tmp
    # Save pycortex volume version of data
    cond_avg_vols[cond_name] = cortex.Volume(tmp, sub, xfm, vmin=-3, vmax=3, cmap='RdBu_r')
    print('Mean response to {} has shape: '.format(cond_name), tmp.shape)
    del tmp

In [ ]:
# Review condition_names
condition_names

In [ ]:
### STUDENT ANSWER (part 2)
# View each event-related averages as a movie
# This is the most compact way to do it:
for condition in condition_names:
    print('Condition: {}'.format(condition))
    cortex.webgl.show(cond_avg_vols[condition], output='none')

Describe what you see! 

**This was graded very leniently.** Most sensible statements got full points. 

Activity appeared to rise and fall in each condition. For some conditions, the activity seemed to change across the entire brain, and to rise and fall multiple times. For other conditions, the activity appeared more localized. For some conditions, contiguous patches or regions of the brain seemed to become active partway through the event average. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
### STUDENT ANSWER
# Use cortex.quickflat.make_figure() to show responses for all voxels at the 5th TR 
# in each event-related average
for condition in condition_names:
    new_vol = cortex.Volume(cond_avg_data[condition][4], sub, xfm, vmin=-3, vmax=3)
    cortex.quickflat.make_figure(new_vol)
    plt.suptitle("Condition = {}, 5th TR".format(condition), fontsize=24)

In [ ]:
### STUDENT ANSWER
# Re-compute event-related averages using 3 runs of data instead of just 1 run
# Load all three runs of data (Same ones that we used for HW last week...)
data_files = ['s01_categories_01.nii.gz', 
              's01_categories_02.nii.gz', 
              's01_categories_03.nii.gz']
data_files = [os.path.join(neurods.io.data_list['fmri'],'categories',f) for f in data_files]
data_full = neurods.io.load_fmri_data(*data_files, mask=cortical_voxels, 
                                      do_zscore=True, dtype=np.float32)
# Load all three runs of the experimental design
design_full = np.vstack([design[run] for run in ['run1', 'run2', 'run3']])

# Same code as above, modified to take `data_full` and `design_full`
# Compute event-related averages for each condition
event_duration = 10
# Set up variable to store condition average(s)
# (These steps are not necessary for full points, but they make everything shorter / cleaner)
# ... for raw average data
cond_avg_data = dict()
# ... for pycortex volumes of condition averages
cond_avg_vols = dict()
for cond_num, cond_name in enumerate(condition_names):
    # Get onset indices
    # Now full design. Onsets will be computed the same way; note that each design array
    # matches up with the time points (TRs) from the corresponding data run this way.
    cond_bool = design_full[:, cond_num] 
    onset_indices = neurods.utils.get_onsets(cond_bool)
    tmp = []
    for oi in onset_indices:
        tmp.append(data_full[oi:oi+event_duration, :])
    tmp = np.array(tmp)
    tmp = np.mean(tmp, axis=0)
    # Save average data
    cond_avg_data[cond_name] = tmp
    # Save pycortex volume version of data
    cond_avg_vols[cond_name] = cortex.Volume(tmp, sub, xfm, vmin=-3, vmax=3, cmap='RdBu_r')
    print('Mean response to {} has shape: '.format(cond_name), tmp.shape)
    del tmp

In [ ]:
# Re-make plots of all voxels at the 5th TR of the event averages using all the data
for condition in condition_names:
    new_vol = cortex.Volume(cond_avg_data[condition][4], sub, xfm, vmin=-3, vmax=3)
    cortex.quickflat.make_figure(new_vol)
    plt.suptitle("Condition = {}, 5th TR".format(condition), fontsize=24)

# Discussion
Yes, the results appear quite different - there now appear to be more clearly distinct regions activated in response to some of the conditions (faces, bodies, and places). The object condition elicits responses from many voxels in the occipital lobe (the back of the brain), and the scrambled condition does not appear to elicit much response at all.

# Hints
Remember the ways we discussed to reduce memory use! Use a mask as you are loading your data!
```python
cortical_voxels = cortex.db.get_mask(sub, xfm, type='cortical')
```

Each answer does not necessarily have to be one cell!

Remember to break up long lines of code! 

```python
def break_lines(up, like, this=so, 
    you=dont, run=off, the=page):
    """F'REALS"""
    print("We can't grade what we can't see!")
    print('It can also make your code much',
        'more readable!')
    return
```

Look back at the lecture to see examples of how to use `cortex.quickflat.make_figure()`. What type of data structure does make_figure() take as input? What shape array do you use to create that data structure? 

For the last part, you will need to make use of the experimental design arrays for all three runs. These are in the `design` variable loaded above: `design['run1']`, `design['run2']`, and `design['run3']`. Refer back to the lecture if you need a refresher on what the values in these arrays mean.

In [ ]:
# Bonus question:

def compute_event_avg(data, design, cond_num, duration=10):
    """Compute an event-related average
    
    [DON'T FORGET A DOCSTRING!]
    
    Parameters
    ----------
    data : array
        array of data for computation of event-related average
    design : array
        binary array of (time x conditions), with ones for times (TRs) when 
        conditions were present and zeros for times when conditions were not present
    cond_num : int
        condition number for which to compute event average. Indexes into columns of
        `design` argument.
    duration : int
        number of time points for which to compute event average. 
        
    Returns
    -------
    event_avg : array
        time x voxels array of average responses to condition `cond_num`
    """
    cond_bool = design[:, cond_num] 
    onset_indices = neurods.utils.get_onsets(cond_bool)
    tmp = []
    for oi in onset_indices:
        tmp.append(data[oi:oi+duration, :])
    tmp = np.array(tmp)
    out = np.mean(tmp, axis=0)
    return out